In [103]:
import torch
import numpy as np
import torch.nn as nn

xr = torch.from_numpy(np.array([1,0,2,0,1,0,0,1,2,1,1,2,1,0,1,0,1,1,1,1,1,2,0,1,1]).reshape(-1, 5))
xg = torch.from_numpy(np.array([0,1,2,1,1,2,1,0,1,1,2,1,1,2,0,0,0,1,2,2,0,0,0,1,2]).reshape(-1, 5))
xb = torch.from_numpy(np.array([1,0,0,1,1,1,1,1,0,0,0,2,2,0,2,0,2,0,1,0,2,2,0,1,1]).reshape(-1, 5))
x = torch.cat([xr, xg, xb]).view(3, 5, 5)
w0 = torch.LongTensor([[-1,-1,0,-1,-1,1,-1,0,1], [0,-1,1,1,0,1,1,1,-1],[0,1,0,-1,1,0,-1,-1,1]]).view(3, 3, 3)

xr = xr.numpy().reshape(-1,)
w0_1 = w0[0].numpy().reshape(-1,)
b = [1]
o = np.convolve(xr, w0_1)
print(xr.shape)


(25,)
34.6410161514


In [147]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.fc4 = nn.Linear(10, 1)
        self.o = nn.Softmin()
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        x = self.o(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        flat_features = 1
        for t in size:
            flat_features *= t
        return flat_features
            
    
net = Net()
input = Variable(torch.randn(1, 1, 32, 32))
output = net(input)

Variable containing:
 1
[torch.FloatTensor of size 1x1]



/Users/shian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for softmin has been deprecated. Change the call to include dim=X as an argument.


In [4]:
from PIL import Image
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
print('hi')
img = Image.open('/Users/Shian/ml/deepbi/test_image.tif')
img = ToTensor()(img)
img = img.view(-1, 1).numpy()
print(img.shape)
num = 0
for t in img:
    if t != 0:
        num += 1
    if t == 300:
        print(t)
print('end')
print(num)

hi
(13682576, 1)
end
1046529


In [5]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

In [7]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = 'hymenoptera_data'
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), 
                            data_transforms[x])
    for x in ['train', 'val']
}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=4,
                                              shuffule=True, num_workers)}